# 4. Topic Modeling

### Analyze transcripts for topics, define topics, see top topics for each president

In [1]:
import pandas as pd
import pickle

# henter vores dataframe (document-term), hvor vores definerede stopwords er sorteret fra
data = pd.read_pickle('dtm_stop.pkl')
data

,aaron,abandon,abandoned,abandoning,abandonment,abhorrent,ability,abject,able,ablebodied,...,young,youre,youth,youthful,youve,zeitchik,zero,zimbabwe,zone,zones
bushjunior,0,1,0,0,0,0,4,0,0,0,...,2,0,0,0,0,0,0,1,0,0
bushsenior,0,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,1,0
carter,0,1,1,0,0,1,2,0,0,0,...,2,0,1,0,0,0,0,0,0,0
clinton,2,0,0,0,0,0,1,0,4,0,...,9,1,0,0,0,0,0,0,0,1
johnson,0,0,0,0,0,0,0,0,10,0,...,2,0,0,0,0,0,0,0,0,0
kennedy,0,1,0,0,1,0,4,0,0,0,...,3,0,2,1,0,0,0,0,0,0
nixon,0,0,0,1,0,0,1,0,4,0,...,2,0,1,0,0,0,1,0,0,0
obama,0,0,0,0,0,0,0,0,1,0,...,3,0,0,0,1,0,0,0,0,0
reagan,0,0,0,0,0,0,0,0,0,0,...,5,0,0,0,0,0,0,0,0,0
trump,0,0,0,0,0,0,0,1,3,0,...,5,0,0,0,0,2,0,0,0,0


In [2]:
from gensim import matutils, models
import scipy.sparse

# For at lave Topic Modeling, skal vi bruge formatet term-document, så vi 'vender' vores df
tdm = data.transpose()
tdm.head()

,bushjunior,bushsenior,carter,clinton,johnson,kennedy,nixon,obama,reagan,trump
aaron,0,0,0,2,0,0,0,0,0,0
abandon,1,0,1,0,0,1,0,0,0,0
abandoned,0,0,1,0,0,0,0,0,0,0
abandoning,0,0,0,0,0,0,1,0,0,0
abandonment,0,0,0,0,0,1,0,0,0,0


In [3]:
# We're going to put the term-document matrix into a new gensim format, from df --> sparse matrix --> gensim corpus
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

In [4]:
# Gensim also requires dictionary of the all terms and their respective location in the term-document matrix
cv = pickle.load(open("cv_stop.pkl", "rb"))
id2word = dict((v, k) for k, v in cv.vocabulary_.items())

In [5]:
# Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term),
# we need to specify two other parameters as well - the number of topics and the number of passes

# Vi prøvede med 4 topics og 80 passes, men faktisk gav flere end 2 topics ikke mening
# lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=4, passes=80)
# lda.print_topics()

[(0,
  '0.005*"work" + 0.004*"security" + 0.004*"soviet" + 0.004*"peace" + 0.004*"future" + 0.004*"nations" + 0.003*"military" + 0.003*"better" + 0.003*"economy" + 0.003*"change"'),
 (1,
  '0.005*"peace" + 0.005*"great" + 0.004*"nations" + 0.004*"act" + 0.004*"vietnam" + 0.004*"budget" + 0.004*"commitments" + 0.003*"increase" + 0.003*"public" + 0.003*"administration"'),
 (2,
  '0.005*"tax" + 0.004*"children" + 0.004*"work" + 0.003*"nations" + 0.003*"home" + 0.003*"health" + 0.003*"billion" + 0.003*"support" + 0.003*"long" + 0.003*"care"'),
 (3,
  '0.007*"peace" + 0.006*"government" + 0.005*"federal" + 0.005*"freedom" + 0.004*"great" + 0.004*"future" + 0.004*"budget" + 0.003*"family" + 0.003*"free" + 0.003*"americas"')]

In [8]:
# DENNE GAV GOD MENING
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=3, passes=80)
lda.print_topics()

[(0,
  '0.005*"work" + 0.003*"future" + 0.003*"economy" + 0.003*"better" + 0.003*"iraq" + 0.003*"way" + 0.003*"jobs" + 0.003*"security" + 0.002*"past" + 0.002*"change"'),
 (1,
  '0.007*"peace" + 0.005*"nations" + 0.004*"great" + 0.004*"work" + 0.003*"budget" + 0.003*"freedom" + 0.003*"children" + 0.003*"federal" + 0.003*"today" + 0.003*"support"'),
 (2,
  '0.004*"health" + 0.003*"tax" + 0.003*"federal" + 0.003*"government" + 0.003*"pay" + 0.003*"things" + 0.003*"economy" + 0.002*"changes" + 0.002*"reform" + 0.002*"family"')]

In [7]:
# Topic 0: Økonomisk politik
# Topic 1: Udenrigspolitik/militær
# Topic 2: Velfærdspolitik

# Let's take a look at which topics each transcript contains
corpus_transformed = lda[corpus]
print(lda)
list(zip([a for [(a,b)] in corpus_transformed], data.index))

LdaModel(num_terms=5519, num_topics=3, decay=0.5, chunksize=2000)


[(1, 'bushjunior'),
 (0, 'bushsenior'),
 (2, 'carter'),
 (0, 'clinton'),
 (0, 'johnson'),
 (1, 'kennedy'),
 (2, 'nixon'),
 (1, 'obama'),
 (2, 'reagan'),
 (0, 'trump')]